In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import csv
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
test = pd.read_csv("/kaggle/input/2023-ntust-practice-of-social-media-analyticshw1/new_test_data.csv")
train = pd.read_csv("/kaggle/input/2023-ntust-practice-of-social-media-analyticshw1/new_train_data.csv")
sample = pd.read_csv("/kaggle/input/2023-ntust-practice-of-social-media-analyticshw1/sample_submit.csv")

In [3]:
train_table = train.drop("label", axis = 1)
train_label = train.label

In [4]:
G = nx.Graph()
with open("/kaggle/input/2023-ntust-practice-of-social-media-analyticshw1/new_train_data.csv", 'r') as f:
  data = csv.reader(f)
  headers = next(data)
  for row in data:
    G.add_node(int(row[0]))
    G.add_node(int(row[1]))
    if int(row[2]) == 1:
      G.add_edge(int(row[0]), int(row[1]))
with open("/kaggle/input/2023-ntust-practice-of-social-media-analyticshw1/new_test_data.csv", 'r') as f:
  data = csv.reader(f)
  headers = next(data)
  for row in data:
    G.add_node(int(row[1]))
    G.add_node(int(row[2]))

In [5]:
cntuple = tuple(map(tuple, train_table.to_numpy()))

In [6]:
jcfeature = nx.jaccard_coefficient(G, cntuple)
adarfeature = nx.adamic_adar_index(G, cntuple)
prefs = nx.preferential_attachment(G, cntuple)
adar = []
jaccard = []
common = []
prefa = []
for u, v, p in adarfeature:
    adar.append(p)
for u, v, p in jcfeature:
    jaccard.append(p)
for x in cntuple:
    commons = sorted(nx.common_neighbors(G, x[0], x[1]))
    common.append(len(commons))
for u, v, p in prefs:
    prefa.append(p)
train_table["adar"] = adar
train_table["jaccard"] = jaccard
train_table["common"] = common
#train_table["preferential"] = prefa
#adding more similarity index might yield worse accuracy
train_table.sample(20)

,node1,node2,adar,jaccard,common
23322,9754,51703,0.00000,0.000000,0
7096,8499,8576,0.00000,0.000000,0
23110,2153,38788,0.00000,0.000000,0
6687,38747,1602,0.00000,0.000000,0
3961,38043,38281,0.00000,0.000000,0
7575,857,38684,0.00000,0.000000,0
17967,5565,6015,0.00000,0.000000,0
16112,38520,38604,0.00000,0.000000,0
17643,8423,8483,0.00000,0.000000,0
21184,30909,34542,0.00000,0.000000,0


In [7]:
testdata = test.drop("node_pair_id", axis = 1)
ttuple = tuple(map(tuple, testdata.to_numpy()))

In [8]:
tjcfeature = nx.jaccard_coefficient(G, ttuple)
tadarfeature = nx.adamic_adar_index(G, ttuple)
tprefs = nx.preferential_attachment(G, ttuple)
tadar = []
tjaccard = []
tcommon = []
tprefa = []
for u, v, p in tadarfeature:
    tadar.append(p)
for u, v, p in tjcfeature:
    tjaccard.append(p)
for x in ttuple:
    commons = sorted(nx.common_neighbors(G, x[0], x[1]))
    tcommon.append(len(commons))
for u, v, p in tprefs:
    tprefa.append(p)
testdata["adar"] = tadar
testdata["jaccard"] = tjaccard
testdata["common"] = tcommon
#testdata["preferential"] = tprefa
#adding more similarity index might yield worse accuracy
testdata.sample(20)

,node1,node2,adar,jaccard,common
708,8568,8584,0.000000,0.000000,0
705,38639,38757,0.000000,0.000000,0
1813,8453,8616,0.000000,0.000000,0
2582,38080,38283,0.352956,0.038462,1
4964,32126,11770,0.000000,0.000000,0
5891,8495,8670,0.000000,0.000000,0
5698,12627,12630,0.000000,0.000000,0
5833,46831,48067,0.000000,0.000000,0
1339,8624,38044,0.000000,0.000000,0
2668,38062,38067,0.000000,0.000000,0


In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(train_table, train_label, test_size=0.3, random_state=7)

In [10]:
x_valid

,node1,node2,adar,jaccard,common
10900,8578,8605,0.00000,0.00,0
1351,38552,38585,0.00000,0.00,0
9378,38611,40030,0.00000,0.00,0
8271,38107,38086,0.45512,0.04,1
14379,16024,22245,0.00000,0.00,0
...,...,...,...,...,...
7144,8541,8552,0.00000,0.00,0
15116,30259,30261,0.00000,0.00,0
19148,1374,28592,0.00000,0.00,0
13771,38123,38524,0.00000,0.00,0


In [11]:
eval_set = [(x_valid, y_valid)]
xg = XGBClassifier(n_estimators = 115, objective='binary:logistic', learning_rate = 0.15, max_depth = 9)
xg.fit(x_train, y_train, eval_set = eval_set)
preds = xg.predict(testdata)
preds

[0]	validation_0-logloss:0.62859
[1]	validation_0-logloss:0.57969
[2]	validation_0-logloss:0.54321
[3]	validation_0-logloss:0.51053
[4]	validation_0-logloss:0.48402
[5]	validation_0-logloss:0.46406
[6]	validation_0-logloss:0.44399
[7]	validation_0-logloss:0.42646
[8]	validation_0-logloss:0.41287
[9]	validation_0-logloss:0.40136
[10]	validation_0-logloss:0.39218
[11]	validation_0-logloss:0.38652
[12]	validation_0-logloss:0.37982
[13]	validation_0-logloss:0.37534
[14]	validation_0-logloss:0.37009
[15]	validation_0-logloss:0.36586
[16]	validation_0-logloss:0.36506
[17]	validation_0-logloss:0.35994
[18]	validation_0-logloss:0.35637
[19]	validation_0-logloss:0.35572
[20]	validation_0-logloss:0.35158
[21]	validation_0-logloss:0.35103
[22]	validation_0-logloss:0.35072
[23]	validation_0-logloss:0.35053
[24]	validation_0-logloss:0.34713
[25]	validation_0-logloss:0.34628
[26]	validation_0-logloss:0.34634
[27]	validation_0-logloss:0.34566
[28]	validation_0-logloss:0.34560
[29]	validation_0-loglos

array([1, 0, 1, ..., 1, 1, 1])

In [12]:
res = test.copy()
res["label"] = preds
res.head()

,node_pair_id,node1,node2,label
0,0,40963,40966,1
1,1,4544,49357,0
2,2,38726,38760,1
3,3,4636,38678,0
4,4,26789,5842,0


In [13]:
submit = sample.copy()
submit = submit.drop("ans", axis = 1)
submit["ans"] = preds
submit.head()

,node_pair_id,ans
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0


In [14]:
submit.to_csv("submit.csv", index = False)